<a href="https://colab.research.google.com/github/Rajalakshmi2025/Phishing-URL-Detection-Using-Deep-Learning-LSTM-Based-Approach-/blob/main/Phishing_Detection_Using_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Phishing Detection Using Deep Learning**

Developed a deep learning–based phishing detection system to classify URLs as phishing or legitimate using sequential pattern learning.

Utilized the LegitPhish dataset (Mendeley Data, 2025) containing over 100,000 labeled URLs to train and evaluate the model.

Implemented a character-level LSTM architecture to capture structural patterns and anomalies within URLs commonly associated with phishing attacks.

Performed:

Data preprocessing and tokenization of URLs

Sequence padding for uniform input length

Train-test splitting and model evaluation

Performance optimization using dropout to prevent overfitting

Achieved high classification accuracy and demonstrated the practical application of deep learning techniques in cybersecurity.

🔧 Tools & Technologies:

Python, TensorFlow/Keras, Google Colab, Pandas, NumPy

develop a deep learning model that can automatically classify a given URL as phishing (malicious) or legitimate (safe) based on its textual structure and patterns.

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("/content/url_features_extracted1.csv")
# Preview data
print(df.head())
print(df.info())
print(df.describe())


                                               URL  url_length  \
0  https://keraekken-loagginnusa.godaddysites.com/          47   
1         https://metamsk01lgiix.godaddysites.com/          40   
2                          http://myglobaltech.in/          23   
3                   http://djtool-for-spotify.com/          30   
4  https://scearmcoommunnlty.com/invent/freind/get          47   

   has_ip_address  dot_count  https_flag  url_entropy  token_count  \
0               0          2           1     4.250669            6   
1               0          2           1     4.196439            6   
2               0          1           0     3.936180            5   
3               0          1           0     3.894740            5   
4               0          1           1     4.143127            7   

   subdomain_count  query_param_count  tld_length  path_length  \
0                1                  1           3            1   
1                1                  1           3 

In [ ]:
df.isnull().sum()

,0
URL,0
ClassLabel,1


In [ ]:
df.dropna(inplace=True)

In [ ]:
df.shape

(101218, 2)

In [ ]:
# Keep only required columns
df = df[['URL', 'ClassLabel']]

df.head()

,URL,ClassLabel
0,https://keraekken-loagginnusa.godaddysites.com/,0.0
1,https://metamsk01lgiix.godaddysites.com/,0.0
2,http://myglobaltech.in/,0.0
3,http://djtool-for-spotify.com/,0.0
4,https://scearmcoommunnlty.com/invent/freind/get,0.0


In [ ]:
df['ClassLabel'].value_counts()  #check class distribution

,count
ClassLabel,
0.0,63678
1.0,37540


Dataset Summary

Total samples: 101,218

Legitimate (0): 63,678

Phishing (1): 37,540

## STEP 2-Data Preprocessing for LSTM

In [ ]:
#Separate features and labels
X = df['URL']
y = df['ClassLabel']

In [ ]:
#Character-Level Tokenization
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(X)

sequences = tokenizer.texts_to_sequences(X)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = 100
X_padded = pad_sequences(sequences, maxlen=max_len) #padding

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_padded,
    y,
    test_size=0.2,
    random_state=42
)

In [ ]:
print(X_train.shape)
print(X_test.shape)

(80974, 100)
(20244, 100)


Training samples: 80,974

Testing samples: 20,244

Each URL converted into sequence length: 100

## STEP 3 — Build the LSTM Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
print("Vocabulary Size:", vocab_size)

Vocabulary Size: 61


In [ ]:
model = Sequential()

model.add(Embedding(input_dim=vocab_size,
                    output_dim=32,
                    input_length=100))

model.add(LSTM(64))

model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))
model.build(input_shape=(None, 100))

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 100, 32)        │         1,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        24,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 26,849 (104.88 KB)

 Trainable params: 26,849 (104.88 KB)

 Non-trainable params: 0 (0.00 B)

# #STEP 4 — Train the Model

In [ ]:
history = model.fit(
    X_train,
    y_train,
    epochs=5,
    batch_size=128,
    validation_data=(X_test, y_test)
)

Epoch 1/5
633/633 ━━━━━━━━━━━━━━━━━━━━ 88s 135ms/step - accuracy: 0.9470 - loss: 0.1232 - val_accuracy: 0.9957 - val_loss: 0.0154
Epoch 2/5
633/633 ━━━━━━━━━━━━━━━━━━━━ 146s 142ms/step - accuracy: 0.9954 - loss: 0.0161 - val_accuracy: 0.9951 - val_loss: 0.0213
Epoch 3/5
633/633 ━━━━━━━━━━━━━━━━━━━━ 88s 139ms/step - accuracy: 0.9958 - loss: 0.0140 - val_accuracy: 0.9960 - val_loss: 0.0127
Epoch 4/5
633/633 ━━━━━━━━━━━━━━━━━━━━ 88s 139ms/step - accuracy: 0.9971 - loss: 0.0100 - val_accuracy: 0.9969 - val_loss: 0.0085
Epoch 5/5
633/633 ━━━━━━━━━━━━━━━━━━━━ 87s 137ms/step - accuracy: 0.9962 - loss: 0.0124 - val_accuracy: 0.9971 - val_loss: 0.0076


## STEP 5 — Evaluate Model Performance

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)

633/633 ━━━━━━━━━━━━━━━━━━━━ 13s 21ms/step - accuracy: 0.9970 - loss: 0.0077
Test Accuracy: 0.9970855712890625


In [ ]:
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

633/633 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[12731    18]
 [   41  7454]]
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     12749
         1.0       1.00      0.99      1.00      7495

    accuracy                           1.00     20244
   macro avg       1.00      1.00      1.00     20244
weighted avg       1.00      1.00      1.00     20244



## Deploymwnt

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

model = Sequential()
model.add(Embedding(input_dim=vocab_size,
                    output_dim=32,
                    input_length=max_len))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.build(input_shape=(None, max_len))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5, batch_size=128)

Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


633/633 ━━━━━━━━━━━━━━━━━━━━ 84s 129ms/step - accuracy: 0.9411 - loss: 0.1249
Epoch 2/5
633/633 ━━━━━━━━━━━━━━━━━━━━ 79s 124ms/step - accuracy: 0.9959 - loss: 0.0127
Epoch 3/5
633/633 ━━━━━━━━━━━━━━━━━━━━ 79s 125ms/step - accuracy: 0.9967 - loss: 0.0102
Epoch 4/5
633/633 ━━━━━━━━━━━━━━━━━━━━ 83s 127ms/step - accuracy: 0.9979 - loss: 0.0071
Epoch 5/5
633/633 ━━━━━━━━━━━━━━━━━━━━ 83s 128ms/step - accuracy: 0.9971 - loss: 0.0095


In [ ]:
import os
import pickle

os.makedirs("model", exist_ok=True)
os.makedirs("tokenizer", exist_ok=True)

model.save("model/phishing_lstm_model.keras")

with open("tokenizer/tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)